In [1]:
import os

In [2]:
%pwd

'c:\\Users\\gopal\\End-to-end-Machine-Learning-with-MLOps-\\research'

In [3]:
os.chdir("../")

In [4]:
import sys
print("Python executable:", sys.executable)
print("Python path:", sys.path[:3])  # First 3 paths

try:
    import MLOps_Project
    print("MLOps_Project imported successfully")
except ImportError as e:
    print("Import error:", e)

Python executable: C:\Users\gopal\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe
Python path: ['C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\\python312.zip', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\\DLLs', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\\Lib']
MLOps_Project imported successfully


In [5]:
%pwd

'c:\\Users\\gopal\\End-to-end-Machine-Learning-with-MLOps-'

In [12]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [17]:
from MLOps_Project.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH
from MLOps_Project.utils.common import read_yaml, create_directories

ImportError: cannot import name 'CONFIG_FILE_PATH' from 'MLOps_Project.constants' (C:\Users\gopal\End-to-end-Machine-Learning-with-MLOps-\src\MLOps_Project\constants\__init__.py)

In [14]:
import sys
print(sys.executable)

!pip install --upgrade ensure

C:\Users\gopal\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe
Defaulting to user installation because normal site-packages is not writeable


In [21]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [19]:
from pathlib import Path

CONFIG_FILE_PATH = Path("config/config.yaml")
PARAMS_FILE_PATH = Path("params.yaml")
SCHEMA_FILE_PATH = Path("schema.yaml")

print("Defined constants manually")

Defined constants manually


In [22]:
import os
import urllib.request as request
import zipfile
from MLOps_Project import logger
from MLOps_Project.utils.common import get_size

In [23]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  

In [25]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2026-01-19 18:51:23,293: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-01-19 18:51:23,308: INFO: common: yaml file: params.yaml loaded successfully]
[2026-01-19 18:51:23,318: INFO: common: yaml file: schema.yaml loaded successfully]
[2026-01-19 18:51:23,326: INFO: common: created directory at: artifacts]
[2026-01-19 18:51:23,329: INFO: common: created directory at: artifacts/data_ingestion]
[2026-01-19 18:51:24,861: INFO: 2366278714: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: F8B4:141E3F:479B77:9AED7A:696E2FD4
Accept-Ranges: bytes
Date: Mon, 19